### Реализация YOLOv5 в исследовательских целях, на базе открытого репозитория https://github.com/ultralytics/yolov5

> Подробнее о [лицензии](https://ultralytics.com/license)

In [ ]:
!pip install imageio-ffmpeg
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

%mkdir /content/input_imgs

import torch
import os,glob
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import Image
import matplotlib.pyplot as plt
import numpy as np
import imageio
import cv2


In [ ]:
url='https://daily-motor.ru/wp-content/uploads/2022/10/b0aad3e80bc94abaaa403067f33a7d40.jpg'
img=imageio.imread(url)
imageio.imwrite('/content/input_imgs/car.png',img)

**Inference imgs**

In [ ]:
!python detect.py --weights yolov5s.pt --img 640 --conf 0.3 --source /content/input_imgs

Результаты по дефолту сохраняются в *runs/detect/exp{prefix_n}*, воспользуемся интерактом для удобства просмотра


In [ ]:
@interact
def show_images(file=os.listdir('runs/detect/')):
    plt.figure(figsize=(15,8))
  
    img_path='runs/detect/'+file + '/'
    try:
      print(os.listdir('runs/detect/'+file)[0])
      plt.imshow(cv2.cvtColor(cv2.imread(img_path+os.listdir('runs/detect/'+file)[0]),cv2.COLOR_RGB2BGR))
    except IndexError as e:
      print('no image')
    except Exception as e:
      print(e)

**Inference video on url**  
Lets see how fast & furious YOLOv5 🏎

In [ ]:
!python detect.py --weights yolov5s.pt --img 640 --conf 0.3 --source https://www.youtube.com/watch?v=KOb8FheEX-I

**Let's see the result**

In [ ]:
import glob
list_vids=[]
for address, dirs, files in os.walk('runs/detect/'):
   if files:
      #for name in files:
      list_vids.extend([x for x in glob.glob(f"{address}/*.mp4")])
list_vids

**Convert mp4 to frames**

In [ ]:
%mkdir '/content/inf_video/'

In [ ]:
video_url = list_vids[0]
vidcap = cv2.VideoCapture(video_url)
success,image = vidcap.read()
fps = vidcap.get(cv2.CAP_PROP_FPS)
count = 0
for i in range(int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))):
  success,image = vidcap.read()
  if i%int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT)//fps)==0:
    cv2.imwrite(f"/content/inf_video/{i}.jpg", image)      
  
  


In [ ]:
sorted_shots=sorted(os.listdir('/content/inf_video/'),key=lambda x: int(os.path.splitext(x)[0]))

In [ ]:
@interact(frame=(0, len(sorted_shots)-1))
def show_frame(frame=0):
  f = plt.figure(figsize=(18,8))
  plt.imshow(cv2.cvtColor(cv2.imread('/content/inf_video/'+sorted_shots[frame]),cv2.COLOR_RGB2BGR))